In [49]:
from dataclasses import dataclass
import math

@dataclass(frozen=True)
class Vector3:
    x: float
    y: float
    z: float

    # Vector addition
    def __add__(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.x + other.x,
            self.y + other.y,
            self.z + other.z
        )

    # Vector subtraction
    def __sub__(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.x - other.x,
            self.y - other.y,
            self.z - other.z
        )

    # Scalar multiplication
    def __mul__(self, scalar: float) -> "Vector3":
        return Vector3(
            self.x * scalar,
            self.y * scalar,
            self.z * scalar
        )
    
    # Scalar division
    def __truediv__(self, scalar: float) -> "Vector3":
        return Vector3(
            self.x / scalar,
            self.y / scalar,
            self.z / scalar
        )

    __rmul__ = __mul__
    __rtruediv__ = __truediv__

    # Dot product
    def dot(self, other: "Vector3") -> float:
        return (
            self.x * other.x +
            self.y * other.y +
            self.z * other.z
        )

    # Cross product
    def cross(self, other: "Vector3") -> "Vector3":
        return Vector3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )

    # Magnitude (length)
    def magnitude(self) -> float:
        return math.sqrt(self.dot(self))

    # Unit vector
    def unit_vector(self) -> "Vector3":
        mag = self.magnitude()
        if mag == 0:
            raise ValueError("Cannot normalize a zero vector")
        return self * (1 / mag)

In [50]:
# Semi Major Axis (a)
def compute_sma(mu:float, energy:float):
    return -mu/(2*energy)


In [51]:
# Eccentricity (e)
import math

def compute_ecc(energy: float, mu: float, h:float):
    return math.sqrt(1+(2*math.pow(h, 2)*energy)/(math.pow(mu, 2)))

In [52]:
# Inclination (i)
import math

def compute_inc(Z_: Vector3, h_: Vector3):
    Z_ = Z_.unit_vector()
    h_ = h_.unit_vector()
    return math.acos(Z_.dot(h_))


In [53]:
# RAAN (capital omega) Ω
import math

def compute_raan(Nx: float, Ny: float):
    return math.atan2(Ny, Nx)

In [54]:
# Argument of Perigeee (omega sub p) ωp
import math

def compute_argp(h_: Vector3, N_: Vector3, B_: Vector3):
    h_ = h_.unit_vector()
    N_ = N_.unit_vector()
    B_ = B_.unit_vector()
    return math.atan2(h_.dot(N_.cross(B_)), N_.dot(B_))



In [55]:
# True Anomaly (nu) ν
import math

def compute_ta(v_: Vector3, r_: Vector3, B_: Vector3):
    cosTa = r_.dot(B_)/(r_.magnitude()*B_.magnitude())
    ta = math.acos(cosTa)
    if r_.dot(v_) < 0:
        ta = 2*math.pi - ta
    return ta

In [56]:
# Period (TP)
import math

def compute_period(a: float, mu: float):
    return 2*math.pi*math.sqrt(math.pow(a, 3)/mu)

In [ ]:
# Apogee (r sub a) ra
def compute_apogee(a: float, e: float):
    return a*(1+e)

In [ ]:
# Perigee (r sub p) rp
def compute_perigee(a: float, e: float):
    return a*(1-e)

In [59]:
# Energy (xi) ξ
import math

def compute_energy(v: float, r: float, mu: float):
    return math.pow(v, 2)/2 - mu/r

In [60]:
# Angular Momentum Vector (h_)
def compute_angular_momentum_vector(r: Vector3, v: Vector3):
    return r.cross(v)

In [61]:
# RAAN Vector (N_) Unitized
def compute_raan_vector(Z_: Vector3, h_: Vector3):
    Z_ = Z_.unit_vector()
    h_ = h_.unit_vector()
    return (Z_.cross(h_)/(Z_.cross(h_).magnitude())).unit_vector()

In [62]:
# Perigee Vector (B_) Unitized
def compute_perigee_vector(mu: float, r_: Vector3, v_: Vector3, h_: Vector3):
    return v_.cross(h_)-mu*(r_/r_.magnitude()).unit_vector()

In [63]:
from dataclasses import dataclass
import math

@dataclass
class Problem:
    name: str
    pos_vector: Vector3
    vel_vector: Vector3

problem_1 = Problem(
    "Vector 1", 
    Vector3(-464836.978606, -6191644.716805, -2961635.481039), 
    Vector3(7322.77235464, 406.01896116, -1910.89281450)
)
problem_2 = Problem(
    "Vector 2", 
    Vector3(572461.711228, -1015437.194396, 7707337.871302), 
    Vector3(-6195.262945, -3575.889650, -5.423283)
)
problem_3 = Problem(
    "Vector 3", 
    Vector3(-5142754.617115,16130814.767566, 20434322.229790), 
    Vector3(-2924.287128, -2303.326264, 1084.798834)
)
problem_4 = Problem(
    "Vector 4", 
    Vector3(-21100299.894024, 36462486.120500, 69117.555126), 
    Vector3(-2664.268125, -1539.996659, 1.834442)
)

problems = [problem_1, problem_2, problem_3, problem_4]

In [64]:
for problem in problems:
    v = problem.vel_vector.magnitude()
    r = problem.pos_vector.magnitude()
    mu_earth = 3.986004418 * math.pow(10, 14) # m^3/sec^2

    xi = compute_energy(v, r, mu_earth)
    a = compute_sma(mu_earth, xi)
    h_ = compute_angular_momentum_vector(problem.pos_vector, problem.vel_vector)
    ecc = compute_ecc(xi, mu_earth, h_.magnitude())
    inc = compute_inc(Vector3(0,0,1), h_)
    inc_deg = math.degrees(inc)
    N_Unit = compute_raan_vector(Vector3(0,0,1), h_)
    raan = compute_raan(N_Unit.x, N_Unit.y)
    raan_deg = math.degrees(raan)
    B_Unit = compute_perigee_vector(mu_earth, problem.pos_vector, problem.vel_vector, h_)
    argp = compute_argp(h_, N_Unit, B_Unit)
    argp_deg = math.degrees(argp)
    ta = compute_ta(problem.vel_vector, problem.pos_vector, B_Unit)
    ta_deg = math.degrees(ta)
    period = compute_period(a, mu_earth)
    apogee = compute_apogee(a, ecc)
    perigee = compute_perigee(a, ecc)

    print(problem.name)
    print(f"{'Position Vector':<30} {problem.pos_vector}")
    print(f"{'Velocity Vector':<30} {problem.vel_vector}\n")
    print("Keplerian Elements for " + problem.name)
    print(f"{'Semi-Major Axis (a):':<30} {a:16.8f} m")
    print(f"{'Eccentricity (e):':<30} {ecc:16.8f}")
    print(f"{'Inclination (inc):':<30} {inc_deg:16.8f} deg")
    print(f"{'RAAN (raan):':<30} {raan_deg:16.8f} deg")
    print(f"{'Argument of Perigee (wp):':<30} {argp_deg:16.8f} deg")
    print(f"{'True Anomaly (nu):':<30} {ta_deg:16.8f} deg")
    print(f"{'Period (TP):':<30} {period:16.8f} sec")
    print(f"{'Perigee (rp):':<30} {perigee:16.8f} m")
    print(f"{'Apogee (ra):':<30} {apogee:16.8f} m")
    print("\n\n\n")

Vector 1
Position Vector                Vector3(x=-464836.978606, y=-6191644.716805, z=-2961635.481039)
Velocity Vector                Vector3(x=7322.77235464, y=406.01896116, z=-1910.8928145)

Keplerian Elements for Vector 1
Semi-Major Axis (a):           6819999.99903084 m
Eccentricity (e):                    0.01000000
Inclination (inc):                  30.00000000 deg
RAAN (raan):                        30.00000000 deg
Argument of Perigee (wp):           29.99999941 deg
True Anomaly (nu):                 209.43319063 deg
Period (TP):                      5605.15391192 sec
Perigee (rp):                  6751799.99904534 m
Apogee (ra):                   6888199.99901634 m




Vector 2
Position Vector                Vector3(x=572461.711228, y=-1015437.194396, z=7707337.871302)
Velocity Vector                Vector3(x=-6195.262945, y=-3575.88965, z=-5.423283)

Keplerian Elements for Vector 2
Semi-Major Axis (a):           7800000.00120126 m
Eccentricity (e):                    0.00100